# <b/> <center><font color='Cyan'>Trady: Financial AI Agent using LangGraph</font></center>

#### <b/> Table of Contents:
##### 1. Loading LLM(Groq Llama 3.1 70B)
##### 2. Defining Tools
##### 3. Binding Tools with LLM Model
##### 4. Defining ReAct | Tool Calling Agent
##### 5. Query the Agent
##### 6. User Interface to Interact with the Model

In [3]:
pip install python-dotenv langchain langchain-groq yfinance graphviz typing-extensions notebook ipython pillow langgraph alpaca-py


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.9/108.9 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 36.6 MB/s eta 0:00:00


In [4]:
from dotenv import find_dotenv, load_dotenv
import os

load_dotenv(find_dotenv())
groq_api_key = os.environ["GROQ_API_KEY"]

## <b/> 1. Loading LLM Model

 * <b/> Installing LangChain Groq Module

In [6]:
from langchain_groq import ChatGroq
from langchain_core.tools import tool
import alpaca_trade_api as tradeapi
import os

# Load Alpaca API credentials from environment variables
ALPACA_API_KEY = os.environ["ALPACA_API_KEY"]
ALPACA_SECRET_KEY = os.environ["ALPACA_SECRET_KEY"]
ALPACA_BASE_URL = "https://paper-api.alpaca.markets"  # Change to live API URL for live trading

api = tradeapi.REST(ALPACA_API_KEY, ALPACA_SECRET_KEY, ALPACA_BASE_URL, api_version='v2')

llama3 = ChatGroq(api_key = groq_api_key, model = "llama-3.1-70b-versatile")

llama3

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x78d54e62a0e0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x78d54e70a890>, model_name='llama-3.1-70b-versatile', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [7]:
llama3.invoke("Hello, how are you Llama 3.1?")

AIMessage(content="Hello, I'm doing well, thank you for asking. I'm Llama, a large language model designed to assist and communicate with users in a helpful and informative way. I'm here to help answer your questions, provide information, and engage in conversation. What's on your mind, and how can I assist you today?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 67, 'prompt_tokens': 47, 'total_tokens': 114, 'completion_time': 0.268, 'prompt_time': 0.010588494, 'queue_time': 0.14517925, 'total_time': 0.278588494}, 'model_name': 'llama-3.1-70b-versatile', 'system_fingerprint': 'fp_b6828be2c9', 'finish_reason': 'stop', 'logprobs': None}, id='run-758e481d-385c-43d7-830d-b8f47da6ef38-0', usage_metadata={'input_tokens': 47, 'output_tokens': 67, 'total_tokens': 114})

## <b/> 2. Defining Tools

In [9]:
from langchain_core.tools import tool, StructuredTool
import yfinance as yf

@tool
def company_address(ticker: str) -> str:
    """
    Returns company address for input ticker.
    e.g. company_address: AAPL
    Returns company address for ticker AAPL which is stock ticker for Apple Inc.
    """
    ticker_obj = yf.Ticker(ticker)
    info = ticker_obj.get_info()

    return " ".join([info[key] for key in ['address1','city','state','zip','country']])

@tool
def fulltime_employees(ticker: str) -> int:
    """
    Returns fulltime employees count for input ticker.
    e.g. company_address: MSFT
    Returns fulltime employees count for ticker MSFT which is stock ticker for Microsoft.
    """
    ticker_obj = yf.Ticker(ticker)
    info = ticker_obj.get_info()

    return info['fullTimeEmployees']

@tool
def last_close_price(ticker: str) -> float:
    """
    Returns last close price for input ticker.
    e.g. company_address: MSFT
    Returns last close price for ticker MSFT which is stock ticker for Microsoft.
    """
    ticker_obj = yf.Ticker(ticker)
    info = ticker_obj.get_info()

    return info['previousClose']

@tool
def EBITDA(ticker: str) -> float:
    """
    Returns EBITDA for input ticker.
    e.g. company_address: AAPL
    Returns EBITDA for ticker AAPL which is stock ticker for Apple Inc.
    """
    ticker_obj = yf.Ticker(ticker)
    info = ticker_obj.get_info()

    return info['ebitda']

@tool
def total_debt(ticker: str) -> float:
    """
    Returns total debt for input ticker.
    e.g. company_address: AAPL
    Returns total debt for ticker AAPL which is stock ticker for Apple Inc.
    """
    ticker_obj = yf.Ticker(ticker)
    info = ticker_obj.get_info()

    return info['totalDebt']

@tool
def total_revenue(ticker: str) -> float:
    """
    Returns total revenue for input ticker.
    e.g. company_address: MSFT
    Returns total revenue for ticker MSFT which is stock ticker for Microsoft.
    """
    ticker_obj = yf.Ticker(ticker)
    info = ticker_obj.get_info()

    return info['totalRevenue']

@tool
def debt_to_equity_ratio(ticker: str) -> float:
    """
    Returns debt to equity ratio for input ticker.
    e.g. company_address: AAPL
    Returns debt to equity ratio for ticker AAPL which is stock ticker for Apple Inc.
    """
    ticker_obj = yf.Ticker(ticker)
    info = ticker_obj.get_info()

    return info['debtToEquity']

@tool
def get_real_time_price(ticker: str) -> float:
    """
    Fetches the real-time price of the specified stock ticker.
    e.g. get_real_time_price('AAPL')
    Returns the current price of the stock.
    """
    try:
        # Get the current price of the stock
        barset = api.get_barset(ticker, 'minute', limit=1)
        current_price = barset[ticker][0].c  # Get the closing price of the latest bar
        return current_price
    except Exception as e:
        return f"Error fetching data: {str(e)}"

tools = [
    company_address,
    fulltime_employees,
    last_close_price,
    EBITDA,
    total_debt,
    total_revenue,
    debt_to_equity_ratio,
    get_real_time_price
]

## <b/> 3. Binding Tools with LLMs

In [10]:
llama3_with_tools = llama3.bind_tools(tools, tool_choice='auto')

In [11]:
ai_msg = llama3_with_tools.invoke('Hey, How are you? And what can you do?')

print(ai_msg.content)
print(ai_msg.tool_calls)

I'm doing well, thank you for asking. I can help you retrieve information about companies using their stock ticker symbols. I have access to various functions that can provide data such as company address, full-time employees count, last close price, EBITDA, total debt, total revenue, and debt to equity ratio.

If you're looking for information about a specific company, please let me know its stock ticker symbol and what data you're interested in, and I'll be happy to help.
[]


In [12]:
ai_msg = llama3_with_tools.invoke("What's the EBITDA of Microsoft?")

print(ai_msg.content)
print(ai_msg.tool_calls)


[{'name': 'EBITDA', 'args': {'ticker': 'MSFT'}, 'id': 'call_8ffr', 'type': 'tool_call'}]


In [13]:
EBITDA.invoke(ai_msg.tool_calls[0]['args'])

136551997440

## <b/> 4. Defining ReAct / Tool Calling Agent

In [14]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import HumanMessage, AnyMessage, SystemMessage, ToolMessage

class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

class ReActAgent:
    def __init__(self, model, tools, system=""):
        self.system = system
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools, tool_choice="auto")

        graph = StateGraph(AgentState)
        graph.add_node("llama3", self.call_llm)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges(
            "llama3",
            self.exists_action,
            {True: "action", False: END}
        )
        graph.add_edge("action", "llama3")
        graph.set_entry_point("llama3")
        self.graph = graph.compile()

    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def call_llm(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling Tool : {t}")
            if not t['name'] in self.tools:
                print(f"\n Tool : {t} does not exist.")
                result = "Incorrect Tool Name, Please retry and select tool from available tools."
            else:
                result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id = t['id'], name = t['name'], content = str(result)))
        print("Tool execution is completed. Back to the model!")
        return {'messages' : results}

In [15]:
prompt = """
You are a smart AI finance assistant called Trady. Use the list of available tools to answer questions if needed.
You are allowed to make multiple calls (either together or in sequence).
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""

agent = ReActAgent(llama3, tools, system=prompt)

## <b/> 5. Using the Agent

In [16]:
messages = [HumanMessage(content="Hey! Whats Your name? And what can you do?")]

result = agent.graph.invoke({'messages': messages})

result['messages'][-1].content

"Hello. My name is Trady, and I'm a smart AI finance assistant. I can provide information about publicly traded companies using their stock tickers. I can look up a company's address, number of full-time employees, last close price, EBITDA, total debt, total revenue, and debt-to-equity ratio. If you have a specific question about a company, feel free to ask, and I'll do my best to help."

In [17]:
messages = [HumanMessage(content="What is the EBITDA of Nvidia?")]

result = agent.graph.invoke({'messages': messages})

Calling Tool : {'name': 'EBITDA', 'args': {'ticker': 'NVDA'}, 'id': 'call_w71g', 'type': 'tool_call'}
Tool execution is completed. Back to the model!


In [18]:
result

{'messages': [HumanMessage(content='What is the EBITDA of Nvidia?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_w71g', 'function': {'arguments': '{"ticker": "NVDA"}', 'name': 'EBITDA'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 960, 'total_tokens': 977, 'completion_time': 0.068, 'prompt_time': 0.218592706, 'queue_time': 0.15998861099999998, 'total_time': 0.286592706}, 'model_name': 'llama-3.1-70b-versatile', 'system_fingerprint': 'fp_5c5d1b5cfb', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-1897a53b-74ea-471a-858f-74633a73ceea-0', tool_calls=[{'name': 'EBITDA', 'args': {'ticker': 'NVDA'}, 'id': 'call_w71g', 'type': 'tool_call'}], usage_metadata={'input_tokens': 960, 'output_tokens': 17, 'total_tokens': 977}),
  ToolMessage(content='61184000000', name='EBITDA', tool_call_id='call_w71g'),
  AIMessage(content='The EBITDA of Nvidia is $61.184 billio

In [19]:
result['messages'][-1].content

'The EBITDA of Nvidia is $61.184 billion.'

In [20]:
messages = [HumanMessage(content="What is the EBITDA and debt-to-equity ratio of Meta?")]

result = agent.graph.invoke({'messages': messages})

Calling Tool : {'name': 'EBITDA', 'args': {'ticker': 'META'}, 'id': 'call_gz4t', 'type': 'tool_call'}
Calling Tool : {'name': 'debt_to_equity_ratio', 'args': {'ticker': 'META'}, 'id': 'call_3pf3', 'type': 'tool_call'}
Tool execution is completed. Back to the model!


In [21]:
result['messages'][-1].content

'The EBITDA of Meta is 79,208,996,864 and the debt-to-equity ratio is 29.811.'

In [25]:
messages = [HumanMessage(content="Compare total revenue of Amazon and Google.")]

result = agent.graph.invoke({'messages': messages})

Calling Tool : {'name': 'total_revenue', 'args': {'ticker': 'AMZN'}, 'id': 'call_1kyj', 'type': 'tool_call'}
Calling Tool : {'name': 'total_revenue', 'args': {'ticker': 'GOOGL'}, 'id': 'call_xvk1', 'type': 'tool_call'}
Tool execution is completed. Back to the model!


In [26]:
result['messages'][-1].content

"Amazon's total revenue is $620,127,977,472, while Google's total revenue is $339,859,013,632. Amazon's total revenue is approximately 1.82 times that of Google."

In [30]:
messages = [HumanMessage(content="Give me the Realtime price of Apple?")]

result = agent.graph.invoke({'messages': messages})

Calling Tool : {'name': 'last_close_price', 'args': {'ticker': 'AAPL'}, 'id': 'call_6gzn', 'type': 'tool_call'} 
Tool execution is completed. Back to the model!


In [31]:
result['messages'][-1].content

The real-time price of Apple (AAPL) is $234.93, with a -0.09% change.


------

In [55]:
import ipywidgets as widgets
from IPython.display import display, clear_output
from langchain_core.messages import HumanMessage

# Create widgets
input_box = widgets.Text(
    description='Query:',
    placeholder='Type your query here...'
)
output_area = widgets.Output()

# Function to handle user input
def handle_query(sender):
    user_input = input_box.value
    if user_input.lower() == 'exit':
        print("Exiting the Financial AI Assistant.")
        return

    with output_area:
        print(f"\nYou: {user_input}")
        messages = [HumanMessage(content=user_input)]
        result = agent.graph.invoke({'messages': messages})
        response = result['messages'][-1].content
        print(f"\nAI: {response}")


In [56]:
# Link the input box to the handler
input_box.on_submit(handle_query)
display(input_box, output_area)

Text(value='', description='Query:', placeholder='Type your query here...')

Output()

Exiting the Financial AI Assistant.
